# Building design matrices with `ModelSpec`

Force rebuild

In [1]:
x=4
import numpy as np, pandas as pd
%load_ext rpy2.ipython

from ISLP import load_data
from ISLP.models import ModelSpec

import statsmodels.api as sm

In [2]:
Carseats = load_data('Carseats')
%R -i Carseats
Carseats.columns

Index(['Sales', 'CompPrice', 'Income', 'Advertising', 'Population', 'Price',
       'ShelveLoc', 'Age', 'Education', 'Urban', 'US'],
      dtype='object')

## Let's break up income into groups

In [3]:
Carseats['OIncome'] = pd.cut(Carseats['Income'], 
                             [0,50,90,200], 
                             labels=['L','M','H'])
Carseats['OIncome']

0      M
1      L
2      L
3      H
4      M
      ..
395    H
396    L
397    L
398    M
399    L
Name: OIncome, Length: 400, dtype: category
Categories (3, object): ['L' < 'M' < 'H']

Let's also create an unordered version

In [4]:
Carseats['UIncome'] = pd.cut(Carseats['Income'], 
                             [0,50,90,200], 
                             labels=['L','M','H'],
                             ordered=False)
Carseats['UIncome']

0      M
1      L
2      L
3      H
4      M
      ..
395    H
396    L
397    L
398    M
399    L
Name: UIncome, Length: 400, dtype: category
Categories (3, object): ['L', 'M', 'H']

## A simple model

In [5]:
design = ModelSpec(['Price', 'Income'])
X = design.fit_transform(Carseats)
X.columns

Index(['intercept', 'Price', 'Income'], dtype='object')

In [6]:
Y = Carseats['Sales']
M = sm.OLS(Y, X).fit()
M.params

intercept    12.661546
Price        -0.052213
Income        0.012829
dtype: float64

## Basic procedure

The design matrix is built by cobbling together a set of columns and possibly transforming them.
A `pd.DataFrame` is essentially a list of columns. One of the first tasks done  in `ModelSpec.fit`
is to inspect a dataframe for column info. The column `ShelveLoc` is categorical:

In [7]:
Carseats['ShelveLoc']

0         Bad
1        Good
2      Medium
3      Medium
4         Bad
        ...  
395      Good
396    Medium
397    Medium
398       Bad
399      Good
Name: ShelveLoc, Length: 400, dtype: category
Categories (3, object): ['Bad', 'Good', 'Medium']

This is recognized by `ModelSpec` in the form of `Column` objects which are just named tuples with two methods
`get_columns` and `fit_encoder`.

In [8]:
design.column_info_['ShelveLoc']

Column(idx='ShelveLoc', name='ShelveLoc', is_categorical=True, is_ordinal=False, columns=('ShelveLoc[Good]', 'ShelveLoc[Medium]'), encoder=Contrast())

It recognized ordinal columns as well.

In [9]:
design.column_info_['OIncome']

Column(idx='OIncome', name='OIncome', is_categorical=True, is_ordinal=True, columns=('OIncome',), encoder=OrdinalEncoder())

In [10]:
income = design.column_info_['Income']
cols, names = income.get_columns(Carseats)
(cols[:4], names)

(array([ 73,  48,  35, 100]), ('Income',))

## Encoding a column

In building a design matrix we must extract columns from our dataframe (or `np.ndarray`). Categorical
variables usually are encoded by several columns, typically one less than the number of categories.
This task is handled by the `encoder` of the `Column`. The encoder must satisfy the `sklearn` transform
model, i.e. `fit` on some array and `transform` on future arrays. The `fit_encoder` method of `Column` fits
its encoder the first time data is passed to it.

In [11]:
shelve = design.column_info_['ShelveLoc']
cols, names = shelve.get_columns(Carseats)
(cols[:4], names)

(array([[0., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.]]),
 ['ShelveLoc[Good]', 'ShelveLoc[Medium]'])

In [12]:
oincome = design.column_info_['OIncome']
oincome.get_columns(Carseats)[0][:4]

array([[2.],
       [1.],
       [1.],
       [0.]])

## The terms

The design matrix consists of several sets of columns. This is managed by the `ModelSpec` through
the `terms` argument which should be a sequence. The elements of `terms` are often
going to be strings (or tuples of strings for interactions, see below) but are converted to a
`Variable` object and stored in the `terms_` of the fitted `ModelSpec`. A `Variable` is just a named tuple.

In [13]:
design.terms

['Price', 'Income']

In [14]:
design.terms_

[Variable(variables=('Price',), name='Price', encoder=None, use_transform=True, pure_columns=True, override_encoder_colnames=False),
 Variable(variables=('Income',), name='Income', encoder=None, use_transform=True, pure_columns=True, override_encoder_colnames=False)]

While each `Column` can itself extract data, they are all promoted to `Variable` to be of a uniform type.  A
`Variable` can also create columns through the `build_columns` method of `ModelSpec`

In [15]:
price = design.terms_[0]
design.build_columns(Carseats, price)

(     Price
 0      120
 1       83
 2       80
 3       97
 4      128
 ..     ...
 395    128
 396    120
 397    159
 398     95
 399    120
 
 [400 rows x 1 columns],
 ['Price'])

Note that `Variable` objects have a tuple of `variables` as well as an `encoder` attribute. The
tuple of `variables` first creates a concatenated dataframe from all corresponding variables and then
is run through `encoder.transform`. The `encoder.fit` method of each `Variable` is run once during 
the call to `ModelSpec.fit`.

In [16]:
from ISLP.models.model_spec import Variable

new_var = Variable(('Price', 'Income', 'UIncome'), name='mynewvar', encoder=None)
design.build_columns(Carseats, new_var)

(     Price  Income  UIncome[L]  UIncome[M]
 0    120.0    73.0         0.0         1.0
 1     83.0    48.0         1.0         0.0
 2     80.0    35.0         1.0         0.0
 3     97.0   100.0         0.0         0.0
 4    128.0    64.0         0.0         1.0
 ..     ...     ...         ...         ...
 395  128.0   108.0         0.0         0.0
 396  120.0    23.0         1.0         0.0
 397  159.0    26.0         1.0         0.0
 398   95.0    79.0         0.0         1.0
 399  120.0    37.0         1.0         0.0
 
 [400 rows x 4 columns],
 ['Price', 'Income', 'UIncome[L]', 'UIncome[M]'])

Let's now transform these columns with an encoder. Within `ModelSpec` we will first build the
arrays above and then call `pca.fit` and finally `pca.transform` within `design.build_columns`.

In [17]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(design.build_columns(Carseats, new_var)[0]) # this is done within `ModelSpec.fit`
pca_var = Variable(('Price', 'Income', 'UIncome'), name='mynewvar', encoder=pca)
design.build_columns(Carseats, pca_var)

/Users/jonathantaylor/miniconda3/envs/islp_test/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


(     mynewvar[0]  mynewvar[1]
 0      -3.608693    -4.853177
 1      15.081506    35.708630
 2      27.422871    40.774250
 3     -33.973209    13.470489
 4       6.567316   -11.290100
 ..           ...          ...
 395   -36.846346   -18.415783
 396    45.741500     3.245602
 397    49.097533   -35.725355
 398   -13.577772    18.845139
 399    31.927566     0.978436
 
 [400 rows x 2 columns],
 ['mynewvar[0]', 'mynewvar[1]'])

The elements of the `variables` attribute may be column identifiers ( `"Price"`), `Column` instances (`price`)
or `Variable` instances (`pca_var`).

In [18]:
fancy_var = Variable(('Price', price, pca_var), name='fancy', encoder=None)
design.build_columns(Carseats, fancy_var)

/Users/jonathantaylor/miniconda3/envs/islp_test/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


(     Price  Price  mynewvar[0]  mynewvar[1]
 0    120.0  120.0    -3.608693    -4.853177
 1     83.0   83.0    15.081506    35.708630
 2     80.0   80.0    27.422871    40.774250
 3     97.0   97.0   -33.973209    13.470489
 4    128.0  128.0     6.567316   -11.290100
 ..     ...    ...          ...          ...
 395  128.0  128.0   -36.846346   -18.415783
 396  120.0  120.0    45.741500     3.245602
 397  159.0  159.0    49.097533   -35.725355
 398   95.0   95.0   -13.577772    18.845139
 399  120.0  120.0    31.927566     0.978436
 
 [400 rows x 4 columns],
 ['Price', 'Price', 'mynewvar[0]', 'mynewvar[1]'])

We can of course run PCA again on these features (if we wanted).

In [19]:
pca2 = PCA(n_components=2)
pca2.fit(design.build_columns(Carseats, fancy_var)[0]) # this is done within `ModelSpec.fit`
pca2_var = Variable(('Price', price, pca_var), name='fancy_pca', encoder=pca2)
design.build_columns(Carseats, pca2_var)

/Users/jonathantaylor/miniconda3/envs/islp_test/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Users/jonathantaylor/miniconda3/envs/islp_test/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Users/jonathantaylor/miniconda3/envs/islp_test/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


(     fancy_pca[0]  fancy_pca[1]
 0       -6.951792      4.859283
 1       55.170148    -24.694875
 2       59.418556    -38.033572
 3       34.722389     28.922184
 4      -21.419184     -3.120673
 ..            ...           ...
 395    -18.257348     40.760122
 396    -10.546709    -45.021658
 397    -77.706359    -37.174379
 398     36.668694      7.730851
 399     -9.540535    -31.059122
 
 [400 rows x 2 columns],
 ['fancy_pca[0]', 'fancy_pca[1]'])

## Building the design matrix

With these notions in mind, the final design is essentially then

In [20]:
X_hand = np.column_stack([design.build_columns(Carseats, v)[0] for v in design.terms_])[:4]

An intercept column is added if `design.intercept` is `True` and if the original argument to `transform` is
a dataframe the index is adjusted accordingly.

In [21]:
design.intercept

True

In [22]:
design.transform(Carseats)[:4]

,intercept,Price,Income
0,1.0,120,73
1,1.0,83,48
2,1.0,80,35
3,1.0,97,100


## Predicting

Constructing the design matrix at any values is carried out by the `transform` method.

In [23]:
new_data = pd.DataFrame({'Price':[10,20], 'Income':[40, 50]})
new_X = design.transform(new_data)
M.get_prediction(new_X).predicted_mean

array([12.65257604, 12.25873428])

In [24]:
%%R -i new_data,Carseats
predict(lm(Sales ~ Price + Income, data=Carseats), new_data)

       0        1 
12.65258 12.25873 


### Difference between using `pd.DataFrame` and `np.ndarray`

If the `terms` only refer to a few columns of the data frame, the `transform` method only needs a dataframe with those columns.

If we had used an `np.ndarray`, the column identifiers would be integers identifying specific columns so,
in order to work correctly, `transform` would need another `np.ndarray` where the columns have the same meaning.

In [25]:
Carseats_np = np.asarray(Carseats[['Price', 'ShelveLoc', 'US', 'Income']])
design_np = ModelSpec([0,3]).fit(Carseats_np)
design_np.transform(Carseats_np)[:4]

array([[1.0, 120, 73],
       [1.0, 83, 48],
       [1.0, 80, 35],
       [1.0, 97, 100]], dtype=object)

The following will fail for hopefully obvious reasons

In [26]:
try:
    new_D = np.zeros((2,2))
    new_D[:,0] = [10,20]
    new_D[:,1] = [40,50]
    M.get_prediction(new_D).predicted_mean
except ValueError as e:
    print(e)

shapes (2,2) and (3,) not aligned: 2 (dim 1) != 3 (dim 0)


Ultimately, `M` expects 3 columns for new predictions because it was fit
with a matrix having 3 columns (the first representing an intercept).

We might be tempted to try as with the `pd.DataFrame` and produce
an `np.ndarray` with only the necessary variables.

In [27]:
try:
    new_X = np.zeros((2,2))
    new_X[:,0] = [10,20]
    new_X[:,1] = [40,50]
    new_D = design_np.transform(new_X)
    M.get_prediction(new_D).predicted_mean
except IndexError as e:
    print(e)

index 3 is out of bounds for axis 1 with size 2


This fails because `design_np` is looking for column `3` from its `terms`:

In [28]:
design_np.terms_

[Variable(variables=(0,), name='0', encoder=None, use_transform=True, pure_columns=True, override_encoder_colnames=False),
 Variable(variables=(3,), name='3', encoder=None, use_transform=True, pure_columns=True, override_encoder_colnames=False)]

However, if we have an `np.ndarray` in which the first column indeed represents `Price` and the fourth indeed
represents `Income` then we can arrive at the correct answer by supplying such the array to `design_np.transform`:

In [29]:
new_X = np.zeros((2,4))
new_X[:,0] = [10,20]
new_X[:,3] = [40,50]
new_D = design_np.transform(new_X)
M.get_prediction(new_D).predicted_mean

array([12.65257604, 12.25873428])

Given this subtlety about needing to supply arrays with identical column structure to `transform` when
using `np.ndarray` we presume that using a `pd.DataFrame` will be the more popular use case.

## A model with some categorical variables

Categorical variables become `Column` instances with encoders.

In [30]:
design = ModelSpec(['Population', 'Price', 'UIncome', 'ShelveLoc']).fit(Carseats)
design.column_info_['UIncome']

Column(idx='UIncome', name='UIncome', is_categorical=True, is_ordinal=False, columns=('UIncome[L]', 'UIncome[M]'), encoder=Contrast())

In [31]:
X = design.fit_transform(Carseats)
X.columns

Index(['intercept', 'Population', 'Price', 'UIncome[L]', 'UIncome[M]',
       'ShelveLoc[Good]', 'ShelveLoc[Medium]'],
      dtype='object')

In [32]:
sm.OLS(Y, X).fit().params

intercept            11.876012
Population            0.001163
Price                -0.055725
UIncome[L]           -1.042297
UIncome[M]           -0.119123
ShelveLoc[Good]       4.999623
ShelveLoc[Medium]     1.964278
dtype: float64

In [33]:
%%R
lm(Sales ~ Population + Price + UIncome + ShelveLoc, data=Carseats)$coef

    (Intercept)      Population           Price        UIncomeM        UIncomeH 
    10.83371503      0.00116301     -0.05572469      0.92317388      1.04229679 
  ShelveLocGood ShelveLocMedium 
     4.99962319      1.96427771 


## Getting the encoding you want

By default the level dropped by `ModelSpec` will be the first of the `categories_` values from 
`sklearn.preprocessing.OneHotEncoder()`. We might wish to change this. It seems
as if the correct way to do this would be something like `Variable(('UIncome',), 'mynewencoding', new_encoder)`
where `new_encoder` would somehow drop the column we want dropped. 

However, when using the convenient identifier `UIncome` in the `variables` argument, this maps to the `Column` associated to `UIncome` within `design.column_info_`:

In [34]:
design.column_info_['UIncome']

Column(idx='UIncome', name='UIncome', is_categorical=True, is_ordinal=False, columns=('UIncome[L]', 'UIncome[M]'), encoder=Contrast())

This column already has an encoder and `Column` instances are immutable as named tuples. Further, there are times when 
we may want to encode `UIncome` differently within the same model. In the model below the main effect of `UIncome` is encoded with two columns while in the interaction `UIncome` (see below) has three columns. This is a design of interest
and we need a way to allow different encodings of the same column of `Carseats`

In [35]:
%%R
lm(Sales ~ UIncome:ShelveLoc + UIncome, data=Carseats)


Call:
lm(formula = Sales ~ UIncome:ShelveLoc + UIncome, data = Carseats)

Coefficients:
             (Intercept)                  UIncomeM                  UIncomeH  
                  5.1317                    0.1151                    1.1561  
  UIncomeL:ShelveLocGood    UIncomeM:ShelveLocGood    UIncomeH:ShelveLocGood  
                  4.5121                    5.5752                    3.7381  
UIncomeL:ShelveLocMedium  UIncomeM:ShelveLocMedium  UIncomeH:ShelveLocMedium  
                  1.2473                    2.4782                    1.5141  



 We can create a new 
`Column` with the encoder we want. For categorical variables, there is a convenience function to do so.

In [36]:
from ISLP.models.model_spec import contrast
pref_encoding = contrast('UIncome', 'drop', 'L')

In [37]:
design.build_columns(Carseats, pref_encoding)

(     UIncome[M]  UIncome[H]
 0           1.0         0.0
 1           0.0         0.0
 2           0.0         0.0
 3           0.0         1.0
 4           1.0         0.0
 ..          ...         ...
 395         0.0         1.0
 396         0.0         0.0
 397         0.0         0.0
 398         1.0         0.0
 399         0.0         0.0
 
 [400 rows x 2 columns],
 ['UIncome[M]', 'UIncome[H]'])

In [38]:
design = ModelSpec(['Population', 'Price', pref_encoding, 'ShelveLoc']).fit(Carseats)
X = design.fit_transform(Carseats)
X.columns

Index(['intercept', 'Population', 'Price', 'UIncome[M]', 'UIncome[H]',
       'ShelveLoc[Good]', 'ShelveLoc[Medium]'],
      dtype='object')

In [39]:
sm.OLS(Y, X).fit().params

intercept            10.833715
Population            0.001163
Price                -0.055725
UIncome[M]            0.923174
UIncome[H]            1.042297
ShelveLoc[Good]       4.999623
ShelveLoc[Medium]     1.964278
dtype: float64

In [40]:
%%R
lm(Sales ~ Population + Price + UIncome + ShelveLoc, data=Carseats)$coef

    (Intercept)      Population           Price        UIncomeM        UIncomeH 
    10.83371503      0.00116301     -0.05572469      0.92317388      1.04229679 
  ShelveLocGood ShelveLocMedium 
     4.99962319      1.96427771 


## Interactions

We've referred to interactions above. These are specified (by convenience) as tuples in the `terms` argument
to `ModelSpec`.

In [41]:
design = ModelSpec([('UIncome', 'ShelveLoc'), 'UIncome'])
X = design.fit_transform(Carseats)
sm.OLS(Y, X).fit().params

intercept                       7.866634
UIncome[L]:ShelveLoc[Good]      4.512054
UIncome[L]:ShelveLoc[Medium]    1.247275
UIncome[M]:ShelveLoc[Good]      5.575170
UIncome[M]:ShelveLoc[Medium]    2.478163
UIncome[L]                     -2.734895
UIncome[M]                     -2.619745
dtype: float64

The tuples in `terms` are converted to `Variable` in the formalized `terms_` attribute by creating a `Variable` with
`variables` set to the tuple and the encoder an `Interaction` encoder which (unsurprisingly) creates the interaction columns from the concatenated data frames of `UIncome` and `ShelveLoc`.

In [42]:
design.terms_[0]

Variable(variables=('UIncome', 'ShelveLoc'), name='UIncome:ShelveLoc', encoder=Interaction(column_names={'ShelveLoc': ['ShelveLoc[Good]', 'ShelveLoc[Medium]'],
                          'UIncome': ['UIncome[L]', 'UIncome[M]']},
            columns={'ShelveLoc': range(2, 4), 'UIncome': range(0, 2)},
            variables=['UIncome', 'ShelveLoc']), use_transform=True, pure_columns=False, override_encoder_colnames=False)

Comparing this to the previous `R` model.

In [43]:
%%R
lm(Sales ~ UIncome:ShelveLoc + UIncome, data=Carseats)


Call:
lm(formula = Sales ~ UIncome:ShelveLoc + UIncome, data = Carseats)

Coefficients:
             (Intercept)                  UIncomeM                  UIncomeH  
                  5.1317                    0.1151                    1.1561  
  UIncomeL:ShelveLocGood    UIncomeM:ShelveLocGood    UIncomeH:ShelveLocGood  
                  4.5121                    5.5752                    3.7381  
UIncomeL:ShelveLocMedium  UIncomeM:ShelveLocMedium  UIncomeH:ShelveLocMedium  
                  1.2473                    2.4782                    1.5141  



We note a few important things:

1. `R` has reorganized the columns of the design from the formula: although we wrote `UIncome:ShelveLoc` first these
columns have been built later. **`ModelSpec` builds columns in the order determined by `terms`!**

2. As noted above, `R` has encoded `UIncome` differently in the main effect and in the interaction. For `ModelSpec`, the reference to `UIncome` always refers to the column in `design.column_info_` and will always build only the columns for `L` and `M`. **`ModelSpec` does no inspection of terms to decide how to encode categorical variables.**

A few notes:

- **Why not try to inspect the terms?** For any nontrivial formula in `R` with several categorical variables and interactions, predicting what columns will be produced from a given formula is not simple. **`ModelSpec` errs on the side of being explicit.**

- **Is it impossible to build the design as `R` has?** No. An advanced user who *knows* they want the columns built as `R` has can do so (fairly) easily.

In [44]:
full_encoding = contrast('UIncome', None)
design.build_columns(Carseats, full_encoding)

(     UIncome[H]  UIncome[L]  UIncome[M]
 0           0.0         0.0         1.0
 1           0.0         1.0         0.0
 2           0.0         1.0         0.0
 3           1.0         0.0         0.0
 4           0.0         0.0         1.0
 ..          ...         ...         ...
 395         1.0         0.0         0.0
 396         0.0         1.0         0.0
 397         0.0         1.0         0.0
 398         0.0         0.0         1.0
 399         0.0         1.0         0.0
 
 [400 rows x 3 columns],
 ['UIncome[H]', 'UIncome[L]', 'UIncome[M]'])

In [45]:
design = ModelSpec([pref_encoding, (full_encoding, 'ShelveLoc')])
X = design.fit_transform(Carseats)
sm.OLS(Y, X).fit().params

intercept                       5.131739
UIncome[M]                      0.115150
UIncome[H]                      1.156118
UIncome[H]:ShelveLoc[Good]      3.738052
UIncome[H]:ShelveLoc[Medium]    1.514104
UIncome[L]:ShelveLoc[Good]      4.512054
UIncome[L]:ShelveLoc[Medium]    1.247275
UIncome[M]:ShelveLoc[Good]      5.575170
UIncome[M]:ShelveLoc[Medium]    2.478163
dtype: float64

## Special encodings

For flexible models, we may want to consider transformations of features, i.e. polynomial
or spline transformations. Given transforms that follow the `fit/transform` paradigm
we can of course achieve this with a `Column` and an `encoder`. The `ISLP.transforms`
package includes a `Poly` transform

In [46]:
from ISLP.models.model_spec import poly
poly('Income', 3)

Variable(variables=('Income',), name='poly(Income, 3, )', encoder=Poly(degree=3), use_transform=True, pure_columns=False, override_encoder_colnames=True)

In [47]:
design = ModelSpec([poly('Income', 3), 'ShelveLoc'])
X = design.fit_transform(Carseats)
sm.OLS(Y, X).fit().params

intercept                5.440077
poly(Income, 3, )[0]    10.036373
poly(Income, 3, )[1]    -2.799156
poly(Income, 3, )[2]     2.399601
ShelveLoc[Good]          4.808133
ShelveLoc[Medium]        1.889533
dtype: float64

Compare:

In [48]:
%%R
lm(Sales ~ poly(Income, 3) + ShelveLoc, data=Carseats)$coef

     (Intercept) poly(Income, 3)1 poly(Income, 3)2 poly(Income, 3)3 
        5.440077        10.036373        -2.799156         2.399601 
   ShelveLocGood  ShelveLocMedium 
        4.808133         1.889533 


## Splines

Support for natural and B-splines is also included

In [49]:
from ISLP.models.model_spec import ns, bs, pca
design = ModelSpec([ns('Income', df=5), 'ShelveLoc'])
X = design.fit_transform(Carseats)
sm.OLS(Y, X).fit().params

intercept                4.240421
ns(Income, , df=5)[0]    1.468196
ns(Income, , df=5)[1]    1.499471
ns(Income, , df=5)[2]    1.152070
ns(Income, , df=5)[3]    2.418398
ns(Income, , df=5)[4]    1.804460
ShelveLoc[Good]          4.810449
ShelveLoc[Medium]        1.881095
dtype: float64

In [50]:
%%R
library(splines)
lm(Sales ~ ns(Income, df=5) + ShelveLoc, data=Carseats)$coef

        (Intercept) ns(Income, df = 5)1 ns(Income, df = 5)2 ns(Income, df = 5)3 
           4.240421            1.468196            1.499471            1.152070 
ns(Income, df = 5)4 ns(Income, df = 5)5       ShelveLocGood     ShelveLocMedium 
           2.418398            1.804460            4.810449            1.881095 


In [51]:
design = ModelSpec([bs('Income', df=7, degree=2), 'ShelveLoc'])
X = design.fit_transform(Carseats)
sm.OLS(Y, X).fit().params

intercept                          3.495085
bs(Income, , df=7, degree=2)[0]    1.813118
bs(Income, , df=7, degree=2)[1]    0.961852
bs(Income, , df=7, degree=2)[2]    2.471545
bs(Income, , df=7, degree=2)[3]    2.158891
bs(Income, , df=7, degree=2)[4]    2.091625
bs(Income, , df=7, degree=2)[5]    2.600669
bs(Income, , df=7, degree=2)[6]    2.843108
ShelveLoc[Good]                    4.804919
ShelveLoc[Medium]                  1.880337
dtype: float64

In [52]:
%%R
lm(Sales ~ bs(Income, df=7, degree=2) + ShelveLoc, data=Carseats)$coef

                    (Intercept) bs(Income, df = 7, degree = 2)1 
                      3.4950851                       1.8131176 
bs(Income, df = 7, degree = 2)2 bs(Income, df = 7, degree = 2)3 
                      0.9618523                       2.4715450 
bs(Income, df = 7, degree = 2)4 bs(Income, df = 7, degree = 2)5 
                      2.1588908                       2.0916252 
bs(Income, df = 7, degree = 2)6 bs(Income, df = 7, degree = 2)7 
                      2.6006694                       2.8431084 
                  ShelveLocGood                 ShelveLocMedium 
                      4.8049190                       1.8803375 


## PCA

In [53]:
design = ModelSpec([pca(['Income', 
                           'Price', 
                           'Advertising', 
                           'Population'], 
                          n_components=2, 
                          name='myvars'), 'ShelveLoc'])
X = design.fit_transform(Carseats)
sm.OLS(Y, X).fit().params

/Users/jonathantaylor/miniconda3/envs/islp_test/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Users/jonathantaylor/miniconda3/envs/islp_test/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Users/jonathantaylor/miniconda3/envs/islp_test/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


intercept                           5.419405
pca(myvars, , n_components=2)[0]   -0.001131
pca(myvars, , n_components=2)[1]   -0.024217
ShelveLoc[Good]                     4.816253
ShelveLoc[Medium]                   1.924139
dtype: float64

In [54]:
%%R
lm(Sales ~ prcomp(cbind(Income, Price, Advertising, Population))$x[,1:2] + ShelveLoc, data=Carseats)


Call:
lm(formula = Sales ~ prcomp(cbind(Income, Price, Advertising, 
    Population))$x[, 1:2] + ShelveLoc, data = Carseats)

Coefficients:
                                                      (Intercept)  
                                                         5.419405  
prcomp(cbind(Income, Price, Advertising, Population))$x[, 1:2]PC1  
                                                         0.001131  
prcomp(cbind(Income, Price, Advertising, Population))$x[, 1:2]PC2  
                                                        -0.024217  
                                                    ShelveLocGood  
                                                         4.816253  
                                                  ShelveLocMedium  
                                                         1.924139  



It is of course common to scale before running PCA.

In [55]:
design = ModelSpec([pca(['Income', 
                           'Price', 
                           'Advertising', 
                           'Population'], 
                          n_components=2, 
                          name='myvars',
                          scale=True), 'ShelveLoc'])
X = design.fit_transform(Carseats)
sm.OLS(Y, X).fit().params

/Users/jonathantaylor/miniconda3/envs/islp_test/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/jonathantaylor/miniconda3/envs/islp_test/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/jonathantaylor/miniconda3/envs/islp_test/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


intercept                           5.352159
pca(myvars, , n_components=2)[0]    0.446383
pca(myvars, , n_components=2)[1]   -1.219788
ShelveLoc[Good]                     4.922780
ShelveLoc[Medium]                   2.005617
dtype: float64

In [56]:
%%R
lm(Sales ~ prcomp(cbind(Income, Price, Advertising, Population), scale=TRUE)$x[,1:2] + ShelveLoc, data=Carseats)


Call:
lm(formula = Sales ~ prcomp(cbind(Income, Price, Advertising, 
    Population), scale = TRUE)$x[, 1:2] + ShelveLoc, data = Carseats)

Coefficients:
                                                                    (Intercept)  
                                                                         5.3522  
prcomp(cbind(Income, Price, Advertising, Population), scale = TRUE)$x[, 1:2]PC1  
                                                                         0.4469  
prcomp(cbind(Income, Price, Advertising, Population), scale = TRUE)$x[, 1:2]PC2  
                                                                        -1.2213  
                                                                  ShelveLocGood  
                                                                         4.9228  
                                                                ShelveLocMedium  
                                                                         2.0056  



There will be some small differences in the coefficients due to `sklearn` use of `np.std(ddof=0)` instead
of `np.std(ddof=1)`.

In [57]:
np.array(sm.OLS(Y, X).fit().params)[1:3] * np.sqrt(X.shape[0] / (X.shape[0]-1))

array([ 0.44694166, -1.22131519])

## Submodels

We can build submodels as well, even if the terms do not appear in the original `terms` argument.
Fundamentally, the terms just need to be able to have the `design.build_columns` work for us to be
able to build a design matrix. The initial inspection of the columns of `Carseats` has created
a column for `US`, hence we can build this submodel.

In [58]:
design = ModelSpec(['UIncome', 'ShelveLoc', 'Price']).fit(Carseats)
design.build_submodel(Carseats, ['US'])

,intercept,US[Yes]
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,0.0
...,...,...
395,1.0,1.0
396,1.0,1.0
397,1.0,1.0
398,1.0,1.0


## ANOVA 

For a given `terms` argument, there as a natural sequence of models, namely those specified by `[terms[:i] for i in range(len(terms)+1]`.

In [59]:
design = ModelSpec(['ShelveLoc', 'Price', 'UIncome', 'US']).fit(Carseats)
for D in design.build_sequence(Carseats):
    print(D.columns)

Index(['intercept'], dtype='object')
Index(['intercept', 'ShelveLoc[Good]', 'ShelveLoc[Medium]'], dtype='object')
Index(['intercept', 'ShelveLoc[Good]', 'ShelveLoc[Medium]', 'Price'], dtype='object')
Index(['intercept', 'ShelveLoc[Good]', 'ShelveLoc[Medium]', 'Price',
       'UIncome[L]', 'UIncome[M]'],
      dtype='object')
Index(['intercept', 'ShelveLoc[Good]', 'ShelveLoc[Medium]', 'Price',
       'UIncome[L]', 'UIncome[M]', 'US[Yes]'],
      dtype='object')


In [60]:
sm.stats.anova_lm(*(sm.OLS(Y, D).fit() for D in design.build_sequence(Carseats) ))

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,399.0,3182.274698,0.0,NaN,NaN,NaN
1,397.0,2172.743555,2.0,1009.531143,153.010858,5.452815e-50
2,396.0,1455.640702,1.0,717.102853,217.377192,1.583751e-39
3,394.0,1378.915938,2.0,76.724764,11.628885,1.239031e-05
4,393.0,1296.462700,1.0,82.453238,24.994257,8.678832e-07


In [61]:
%%R
anova(lm(Sales ~ ShelveLoc + Price + UIncome + US, data=Carseats))

Analysis of Variance Table

Response: Sales
           Df  Sum Sq Mean Sq F value    Pr(>F)    
ShelveLoc   2 1009.53  504.77 153.011 < 2.2e-16 ***
Price       1  717.10  717.10 217.377 < 2.2e-16 ***
UIncome     2   76.72   38.36  11.629 1.240e-05 ***
US          1   82.45   82.45  24.994 8.679e-07 ***
Residuals 393 1296.46    3.30                      
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Recall that `ModelSpec` does not inspect `terms` to reorder based on degree of 
interaction as `R` does:

In [62]:
design = ModelSpec([(full_encoding, 'ShelveLoc'), pref_encoding]).fit(Carseats)
sm.stats.anova_lm(*(sm.OLS(Y, D).fit() for D in design.build_sequence(Carseats) ))

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,399.0,3182.274698,0.0,NaN,NaN,NaN
1,393.0,2059.376413,6.0,1122.898284,35.940047,1.175738e-34
2,391.0,2036.044596,2.0,23.331817,2.240310,1.077900e-01


In [63]:
%%R
anova(lm(Sales ~ UIncome:ShelveLoc + UIncome, data=Carseats))

Analysis of Variance Table

Response: Sales
                   Df  Sum Sq Mean Sq F value    Pr(>F)    
UIncome             2   61.92  30.962  5.9458  0.002859 ** 
UIncome:ShelveLoc   6 1084.31 180.718 34.7049 < 2.2e-16 ***
Residuals         391 2036.04   5.207                      
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


To agree with `R` we must order `terms` as `R` will.

In [64]:
design = ModelSpec([pref_encoding, (full_encoding, 'ShelveLoc')]).fit(Carseats)
sm.stats.anova_lm(*(sm.OLS(Y, D).fit() for D in design.build_sequence(Carseats)))

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,399.0,3182.274698,0.0,NaN,NaN,NaN
1,397.0,3120.351382,2.0,61.923316,5.945846,2.855424e-03
2,391.0,2036.044596,6.0,1084.306785,34.704868,1.346561e-33


## More complicated interactions

Can we have an interaction of a polynomial effect with a categorical? Absolutely

In [65]:
%%R
anova(lm(Sales ~ UIncome + poly(Income, 3):UIncome + UIncome:US, data=Carseats))

Analysis of Variance Table

Response: Sales
                         Df  Sum Sq Mean Sq F value  Pr(>F)  
UIncome                   2   61.92 30.9617  4.0310 0.01851 *
UIncome:poly(Income, 3)   9   79.72  8.8581  1.1533 0.32408  
UIncome:US                3   83.51 27.8367  3.6242 0.01324 *
Residuals               385 2957.12  7.6808                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


To match `R` we note that it has used its inspection rules to encode `UIncome` with 3 levels
for the two interactions.

In [66]:
p3 = poly('Income', 3)
design = ModelSpec([pref_encoding, (p3, full_encoding), (full_encoding, 'US')]).fit(Carseats)
X = design.transform(Carseats)
sm.OLS(Y, X).fit().params

intercept                            65.978856
UIncome[M]                          -60.159607
UIncome[H]                         -147.276154
poly(Income, 3, )[0]:UIncome[H]    1957.694387
poly(Income, 3, )[0]:UIncome[L]    1462.060650
poly(Income, 3, )[0]:UIncome[M]      83.035153
poly(Income, 3, )[1]:UIncome[H]    -984.494570
poly(Income, 3, )[1]:UIncome[L]     881.537647
poly(Income, 3, )[1]:UIncome[M]     -18.006234
poly(Income, 3, )[2]:UIncome[H]     207.614692
poly(Income, 3, )[2]:UIncome[L]     217.190749
poly(Income, 3, )[2]:UIncome[M]      34.065434
UIncome[H]:US                         0.903404
UIncome[L]:US                         0.895538
UIncome[M]:US                         1.048728
dtype: float64

In [67]:
sm.stats.anova_lm(*(sm.OLS(Y, D).fit() for D in design.build_sequence(Carseats)))

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,399.0,3182.274698,0.0,NaN,NaN,NaN
1,397.0,3120.351382,2.0,61.923316,4.031032,0.018488
2,388.0,3040.628559,9.0,79.722823,1.153273,0.324049
3,385.0,2957.118444,3.0,83.510115,3.624181,0.013244


## Grouping columns for ANOVA

The `Variable` construct can be used to group
variables together to get custom sequences of models for `anova_lm`.

In [68]:
group1 = Variable(('Price', pref_encoding), 'group1', None)
group2 = Variable(('US', 'Advertising'), 'group2', None)
design = ModelSpec([group1, group2]).fit(Carseats)
for D in design.build_sequence(Carseats):
    print(D.columns)

Index(['intercept'], dtype='object')
Index(['intercept', 'Price', 'UIncome[M]', 'UIncome[H]'], dtype='object')
Index(['intercept', 'Price', 'UIncome[M]', 'UIncome[H]', 'US[Yes]',
       'Advertising'],
      dtype='object')


In [69]:
sm.stats.anova_lm(*(sm.OLS(Y, D).fit() for D in design.build_sequence(Carseats)))

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,399.0,3182.274698,0.0,NaN,NaN,NaN
1,396.0,2508.187788,3.0,674.086910,39.304841,2.970412e-22
2,394.0,2252.396343,2.0,255.791445,22.372135,6.267562e-10


It is not clear this is simple to do in `R` as the formula object expands all parentheses.

In [70]:
%%R
anova(lm(Sales ~ (Price + UIncome) + (US + Advertising), data=Carseats))

Analysis of Variance Table

Response: Sales
             Df  Sum Sq Mean Sq  F value    Pr(>F)    
Price         1  630.03  630.03 110.2079 < 2.2e-16 ***
UIncome       2   44.06   22.03   3.8533   0.02201 *  
US            1  121.88  121.88  21.3196 5.270e-06 ***
Advertising   1  133.91  133.91  23.4247 1.868e-06 ***
Residuals   394 2252.40    5.72                       
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


It can be done by building up the models
by hand and likely is possible to be done programmatically but it seems not obvious.

In [71]:
%%R
M1 = lm(Sales ~ 1, data=Carseats)
M2 = lm(Sales ~ Price + UIncome, data=Carseats)
M3 = lm(Sales ~ Price + UIncome + US + Advertising, data=Carseats)
anova(M1, M2, M3)

Analysis of Variance Table

Model 1: Sales ~ 1
Model 2: Sales ~ Price + UIncome
Model 3: Sales ~ Price + UIncome + US + Advertising
  Res.Df    RSS Df Sum of Sq      F    Pr(>F)    
1    399 3182.3                                  
2    396 2508.2  3    674.09 39.305 < 2.2e-16 ***
3    394 2252.4  2    255.79 22.372 6.268e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Alternative anova

Another common ANOVA table involves dropping each term in succession from the model and comparing
to the full model.

In [72]:
Dfull = design.transform(Carseats)
Mfull = sm.OLS(Y, Dfull).fit()
for i, D in enumerate(design.build_sequence(Carseats, anova_type='drop')):
    if i == 0:
        D0 = D
    print(set(D.columns) ^ set(Dfull.columns))
    print(sm.stats.anova_lm(sm.OLS(Y, D).fit(), Mfull))

{'intercept'}
   df_resid          ssr  df_diff      ss_diff           F        Pr(>F)
0     395.0  4417.273517      0.0          NaN         NaN           NaN
1     394.0  2252.396343      1.0  2164.877175  378.690726  1.359177e-59
{'Price', 'UIncome[H]', 'UIncome[M]'}
   df_resid          ssr  df_diff     ss_diff          F        Pr(>F)
0     397.0  2950.808154      0.0         NaN        NaN           NaN
1     394.0  2252.396343      3.0  698.411811  40.723184  6.077848e-23
{'US[Yes]', 'Advertising'}
   df_resid          ssr  df_diff     ss_diff          F        Pr(>F)
0     396.0  2508.187788      0.0         NaN        NaN           NaN
1     394.0  2252.396343      2.0  255.791445  22.372135  6.267562e-10


In [73]:
%%R
M1 = lm(Sales ~ Price + UIncome + US + Advertising, data=Carseats)
M2 = lm(Sales ~ US + Advertising, data=Carseats)
print(anova(M2, M1))
M3 = lm(Sales ~ Price + UIncome, data=Carseats)
print(anova(M3, M1))

Analysis of Variance Table

Model 1: Sales ~ US + Advertising
Model 2: Sales ~ Price + UIncome + US + Advertising
  Res.Df    RSS Df Sum of Sq      F    Pr(>F)    
1    397 2950.8                                  
2    394 2252.4  3    698.41 40.723 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
Analysis of Variance Table

Model 1: Sales ~ Price + UIncome
Model 2: Sales ~ Price + UIncome + US + Advertising
  Res.Df    RSS Df Sum of Sq      F    Pr(>F)    
1    396 2508.2                                  
2    394 2252.4  2    255.79 22.372 6.268e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


The comparison without the intercept here is actually very hard to achieve in `R` with `anova` due to its inspection
of the formula.

In [74]:
%%R
M1 = lm(Sales ~ Price + UIncome + US + Advertising, data=Carseats)
M4 = lm(Sales ~ Price + UIncome + US + Advertising - 1, data=Carseats)
print(anova(M4, M1))

Analysis of Variance Table

Model 1: Sales ~ Price + UIncome + US + Advertising - 1
Model 2: Sales ~ Price + UIncome + US + Advertising
  Res.Df    RSS Df  Sum of Sq F Pr(>F)
1    394 2252.4                       
2    394 2252.4  0 9.0949e-13         


It can be found with `summary`.

In [75]:
%%R
summary(M1)


Call:
lm(formula = Sales ~ Price + UIncome + US + Advertising, data = Carseats)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.4437 -1.6351 -0.0932  1.4920  6.8076 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 12.520356   0.643390  19.460  < 2e-16 ***
Price       -0.054000   0.005072 -10.647  < 2e-16 ***
UIncomeM     0.548906   0.281693   1.949   0.0521 .  
UIncomeH     0.708219   0.322028   2.199   0.0284 *  
USYes        0.024181   0.343246   0.070   0.9439    
Advertising  0.119509   0.024692   4.840 1.87e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2.391 on 394 degrees of freedom
Multiple R-squared:  0.2922,	Adjusted R-squared:  0.2832 
F-statistic: 32.53 on 5 and 394 DF,  p-value: < 2.2e-16



In [76]:
378.690726, 19.46**2

(378.690726, 378.69160000000005)